In [1]:
from linares_plot import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import argparse
import csv
import numpy
import matplotlib
from matplotlib import pyplot, image

In [2]:

def gaussian(x, sx, y=None, sy=None):
    """Returns an array of numpy arrays (a matrix) containing values between
    1 and 0 in a 2D Gaussian distribution

    arguments
    x		-- width in pixels
    sx		-- width standard deviation

    keyword argments
    y		-- height in pixels (default = x)
    sy		-- height standard deviation (default = sx)
    """

    # square Gaussian if only x values are passed
    if y == None:
        y = x
    if sy == None:
        sy = sx
    # centers
    xo = x / 2
    yo = y / 2
    # matrix of zeros
    M = numpy.zeros([x, y], dtype=float)
    # gaussian matrix
    for i in range(y):
        for j in range(x):
            M[j, i] = numpy.exp(
                -1.0 * (((float(i) - xo) ** 2 / (2 * sx * sx)) + ((float(j) - yo) ** 2 / (2 * sy * sy))))

    return M





In [3]:
def draw_display(dispsize, imagefile=None):
    """Returns a matplotlib.pyplot Figure and its axes, with a size of
    dispsize, a black background colour, and optionally with an image drawn
    onto it

    arguments

    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)

    keyword arguments

    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)

    returns
    fig, ax		-	matplotlib.pyplot Figure and its axes: field of zeros
                    with a size of dispsize, and an image drawn onto it
                    if an imagefile was passed
    """

    # construct screen (black background)
    screen = numpy.zeros((dispsize[1], dispsize[0], 3), dtype='float32')
    # if an image location has been passed, draw the image
    if imagefile != None:
        # check if the path to the image exists
        if not os.path.isfile(imagefile):
            raise Exception("ERROR in draw_display: imagefile not found at '%s'" % imagefile)
        # load image
        img = mpimg.imread(imagefile)
        #img = image.imread(imagefile)

        # width and height of the image
        w, h = len(img[0]), len(img)
        # x and y position of the image on the display
        x = dispsize[0] / 2 - w / 2
        y = dispsize[1] / 2 - h / 2
        # draw the image on the screen
        screen[int(y):int(y) + h, int(x):int(x) + w, :] += img
    # dots per inch
    dpi = 100.0
    # determine the figure size in inches
    figsize = (dispsize[0] / dpi, dispsize[1] / dpi)
    # create a figure
    fig = pyplot.figure(figsize=figsize, dpi=dpi, frameon=False)
    ax = pyplot.Axes(fig, [0, 0, 1, 1])
    ax.set_axis_off()
    fig.add_axes(ax)
    # plot display
    ax.axis([0, dispsize[0], 0, dispsize[1]])
    ax.imshow(screen)  # , origin='upper')

    return fig, ax


In [ ]:

def fixations_heatmap(df_gazepoints, dispsize, imagefile=None, alpha=0.5, savefilename=None, 
                 gaussianwh=200, plot_xy=False, gaussiansd=None):
    """Draws a heatmap of the provided fixations, optionally drawn over an
    image, and optionally allocating more weight to fixations with a higher
    duration.

    arguments

    gazepoints		-	a list of gazepoint tuples (x, y)
    
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)

    keyword arguments

    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    alpha		-	float between 0 and 1, indicating the transparancy of
                    the heatmap, where 0 is completely transparant and 1
                    is completely untransparant (default = 0.5)
    savefilename	-	full path to the file in which the heatmap should be
                    saved, or None to not save the file (default = None)

    returns

    fig			-	a matplotlib.pyplot Figure instance, containing the
                    heatmap
    """
    
    ###
    gwh = gaussianwh
    gsdwh = gwh / 6 if (gaussiansd is None) else gaussiansd
    #
    gaus = gaussian(gwh, gsdwh)
    # matrix of zeroes
    strt = gwh / 2
    heatmapsize = (dispsize[0], dispsize[1])
    heatmap = numpy.zeros(heatmapsize, dtype=float)
    #
    gazepoints = [(df_gazepoints.X.iloc[i], df_gazepoints.Y.iloc[i]) for i in range(len(df_gazepoints))]
    gaus_standard = gaussian(x=gwh, sx=gsdwh)
    ###
    for i in range(0, len(gazepoints)):
        # get x and y coordinates
        x =  gazepoints[i][0] 
        #print(x)
        y =  gazepoints[i][1] 
        # add Gaussian to the current heatmap
        ####
        ####
        if (x>0) and (y>0) and (x<dispsize[0]) and (y<dispsize[1]):
            x_inf = x - (gwh/2)
            x_sup = x + (gwh/2)
            y_inf = y - (gwh/2)
            y_sup = y + (gwh/2)
            ####
            ####
            if x_inf<0:
                x_inf=0
            if x_sup>dispsize[0]:
                x_sup = dispsize[0]
            if y_inf<0:
                y_inf=0
            if y_sup>dispsize[1]:
                y_sup = dispsize[1]
            ####
            ####
            y_inf=int(y_inf)
            y_sup=int(y_sup)
            x_inf=int(x_inf)
            x_sup=int(x_sup)
            ##
            shape_gauss = (x_sup-x_inf, y_sup-y_inf)
            if shape_gauss==(gwh, gwh):
                gaus = gaus_standard
                heatmap[x_inf:x_sup, y_inf:y_sup ] += gaus
            else:
                gaus = gaussian(x=x_sup-x_inf, y=y_sup-y_inf, sx=gsdwh)
                heatmap[x_inf:x_sup, y_inf:y_sup ] += gaus

            #heatmap[y_inf:y_sup, x_inf:x_sup ] += gaus
            ###
        ###
    ###
    ###
    ###
    heatmap_low = heatmap.copy()
    fig, ax = draw_display(dispsize, imagefile=imagefile)
    lowbound = numpy.mean(heatmap[heatmap > 0])
    heatmap_low[heatmap_low < lowbound] = numpy.NaN
    # draw heatmap on top of image
    ax.imshow(heatmap_low.T, cmap='jet', alpha=alpha) #, origin="lower")
    if plot_xy ==True:
        plt.plot(df_gazepoints.X, df_gazepoints.Y, '--', color='darkred', alpha=0.5, marker='o', linewidth=1, markersize=4)
    # FINISH PLOT
    # invert the y axis, as (0,0) is top left on a display
    ax.invert_yaxis()
    ###
    if savefilename != None:
        fig.savefig(savefilename)
    ###
    